# ELEC3305 - Real Time Sonar App


In [5]:
!pip install pyaudio
# Import functions and libraries
import numpy as np
from scipy import signal
from numpy import *
import matplotlib.pyplot as plt
from rtsonar import rtsonar

### Task III: Play with different parameters with the real-time sonar!

The function `rtsonar()` in `rtsonar.py` provides a wrapper to create a real-time sonar plot using user-defined processing functions. Copy and paste your functions genPulseTrain(), genChirpPulse(), crossCorr(), findDelay(), and dist2time() to the code cell below. Scroll down to the bottom and you should be able to run `rtsonar()`!

The detection resolution and sensitivity will depend on the parameters you will choose for the sonar. The longer the pulse is, the worse time resolution you will get, but much stronger matched filtering amplitude. You can buy the time resolution by using pulse compression with chirp pulses at the expense of increasing the bandwidth. 

* Run the sonar with different parameters. You can get interesting targets by moving your laptop, moving yourself next to the computer, using a book as a reflecting surface. It would be easier for you to distinguish the target if you move it back and forth. Play with the pulse lengths, the frequency sweeps, etc. 
* To get the best real-time performance, you should restart the kernel every time you run this lab. 
* Submit 4 interesting set of parameters. Explain why you chose them. 




In [6]:
# Copy and paste your 5 functions here:
# genChirpPulse()
def genChirpPulse(Npulse, f0, f1, fs):
    #     Function generates an analytic function of a chirp pulse
    #     Inputs:
    #             Npulse - pulse length in samples
    #             f0     - starting frequency of chirp
    #             f1     - end frequency of chirp
    #             fs     - sampling frequency
    #     Output:
    #              pulse - chirp pulse
    t = np.r_[0:Npulse]/fs
    
    T = Npulse / fs
    
    phi_of_t = 2 * np.pi * (f0 * t + (f1-f0) * t**2 / (2*T))
    
    pulse = np.exp(1j * phi_of_t)
    pulse = np.reshape(pulse, (Npulse,1))
    return pulse



    
# genPulseTrain()
def genPulseTrain(pulse, Nrep, Nseg):
    # Funtion generates a pulse train from a pulse. 
    #Inputs:
    #    pulse = the pulse generated by genChirpPulse
    #    Nrep  =  number of pulse repetitions
    #    Nseg  =  Length of pulse segment >= length(pulse)
    pulse_seg = np.append(pulse, np.zeros(Nseg - len(pulse)))
    return np.tile(pulse_seg, Nrep)

    
# crossCorr()
def crossCorr( rcv, pulse_a ):
    # Funtion generates cross-correlation between rcv and pulse_a
    # Inputs:
    #    rcv - received signal
    #    pulse_a - analytic pulse
    # Output:
    #    Xrcv - cross-correlation between rcv and pulse_a
    rcv = np.reshape(rcv, (len(rcv), 1))
    Xrcv = signal.fftconvolve(rcv, pulse_a[::-1])
    return Xrcv
    


# findDelay()
def findDelay(Xrcv, Nseg):
    # finds the first pulse
    # Inputs:  
    #         Xrcv - the received matched filtered signal
    #         Nseg - length of a segment
    # Output:
    #          idx - index of the beginning of the first pulse
    val = np.max(abs(Xrcv))*0.9
    idx = np.argmax(abs(Xrcv)>val)
    return idx


# dist2time()
def dist2time( dist, temperature=21):
    # Converts distance in cm to time in secs
    # Inputs:
    # dist        - distance to object in cm
    # temperature - in celcius
    # Output:
    # t           - time in seconds between transmitted pulse and echo
    v_s = 331.5 * np.sqrt(1 + temperature / 273.15)
    return 2 * dist / (100*v_s)


In [7]:
# Run this for Real-time Sonar
# Change the parameters!
# Your functions will need to be efficient for this to work. 
# If you get nothing try thinking about how to speed up your code.
fs = 96000 # Sampling frequency
f0 = 10000 # Chirp initial frequency
f1 = 20000 # Chirp ending frequency

Npulse = 300 # Length of Chirp Pulse
Nrep = 12 # Number of repetition in a pulse train (determines vertical length of plot )
Nseg = 2048*2 # Number of samples between pulses (determines maximum time-of-arrival)
Nplot = 180 # Number of pixels in plot along horizontal direction (higher resolution is slower)
maxdist = 180 # Maximum distance in cm
temperature = 23 # Temperature     


functions = (genChirpPulse, genPulseTrain, crossCorr, findDelay, dist2time) #join the functions together

stop_flag = rtsonar( f0, f1, fs, Npulse, Nseg, Nrep, Nplot, maxdist, temperature, functions )

In [8]:
# Run this to stop the sonar (it will take a few seconds to stop)
stop_flag.set()